## 用Mnist数据集进行分类算法

In [1]:
# 防止内核挂掉
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

In [2]:
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

导入数据集

In [3]:
# 训练集
train_dataset = datasets.MNIST(root='./data/', 
                               train=True, 
                               transform=transforms.ToTensor(), 
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data/', 
                               train=False, 
                               transform=transforms.ToTensor(), 
                               download=True)

批次设置

In [4]:
# 批次大小
batch_size = 64 # 每次传入数据的量

# 装载训练集
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True) # 打乱数据集

# 装载测试集
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

查看第一个loader中的数据

In [5]:
inputs, labels = next(iter(train_loader))
print('input shape: ', inputs.shape)
print('label shape: ', labels.shape)
print('labels: ', labels)

input shape:  torch.Size([64, 1, 28, 28])
label shape:  torch.Size([64])
labels:  tensor([7, 5, 4, 0, 5, 6, 0, 1, 4, 6, 6, 3, 1, 9, 3, 6, 4, 6, 3, 9, 1, 6, 9, 9,
        7, 2, 5, 1, 4, 8, 7, 3, 3, 0, 8, 7, 8, 0, 8, 6, 2, 7, 5, 6, 8, 3, 2, 5,
        3, 2, 2, 1, 1, 8, 7, 9, 7, 3, 7, 3, 7, 3, 1, 1])


### 网络结构

使用神经网络的框架

In [6]:
class LSTM_Net(nn.Module):
    # 定义网络结构
    def __init__(self):
        super(LSTM_Net, self).__init__()
        self.lstm = nn.LSTM(input_size=28, # 输入特征的大小
                            hidden_size=64, # LSTM模块的数量
                            num_layers=1, # 隐藏层层数
                            batch_first=True) # 数据格式，设置input与output为(batch, seq_len, feature)
        # 全连接层
        self.out = nn.Linear(in_features=64, out_features=10)
        # 激活函数
        self.softmax = nn.Softmax(dim=1)
       
    # 定义网络计算
    def forward(self, x):
        # (batch, seq_len, feature)
        x = x.view(-1, 28, 28)
        ## output: [batch, seq_len, hidden_size] 包含每个序列的输出结果
        ## h_n, c_n: [num_layers, batch, hidden_size] 只包含最后一个序列的输出
        output, (h_n, c_n) = self.lstm(x)
        output_in_last_timestep = h_n[-1, :, :]
        
        z = self.out(output_in_last_timestep)
        output = self.softmax(z)
        
        return output

### 模型参数

In [7]:
# 定义模型
model = LSTM_Net()

# 损失函数
crossentropy_loss = nn.CrossEntropyLoss()

# 优化器
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 训练与测试的方法

In [8]:
# 训练的方法
def train():
    # 模型的训练状态
    # 模型会Dropout
    model.train()
    for i, data in enumerate(train_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 计算loss
        loss = crossentropy_loss(out, labels) # 交叉熵不用one-hot编码
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

In [9]:
# 测试的方法
def test():
    # 模型的验证测试状态
    # 模型不会Dropout
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 获得最大值与其所在位置
        _, predicted = torch.max(out, 1)
        # 预测正确的个数
        correct += (predicted == labels).sum()
    
    print(f'Test acc: {correct.item()/len(test_dataset)}')

    correct = 0
    for i, data in enumerate(train_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 获得最大值与其所在位置
        _, predicted = torch.max(out, 1)
        # 预测正确的个数
        correct += (predicted == labels).sum()
    
    print(f'Train acc: {correct.item()/len(train_dataset)}')

In [10]:
for epoch in range(10):
    print(f'epoch: {epoch}')
    train()
    test()

epoch: 0
Test acc: 0.8399
Train acc: 0.8345833333333333
epoch: 1
Test acc: 0.8679
Train acc: 0.86665
epoch: 2
Test acc: 0.9525
Train acc: 0.9514166666666667
epoch: 3
Test acc: 0.9603
Train acc: 0.9625833333333333
epoch: 4
Test acc: 0.9633
Train acc: 0.9636666666666667
epoch: 5
Test acc: 0.9673
Train acc: 0.9681833333333333
epoch: 6
Test acc: 0.9652
Train acc: 0.9691666666666666
epoch: 7
Test acc: 0.953
Train acc: 0.9563333333333334
epoch: 8
Test acc: 0.9688
Train acc: 0.9729
epoch: 9
Test acc: 0.967
Train acc: 0.9692333333333333
